This notebook trains the bert-base-uncased model (BertModelWithHeads) with the sst-dataset.

In [ ]:
!pip install -U git+https://github.com/Adapter-Hub/adapter-transformers.git
!pip install datasets

  Cloning https://github.com/Adapter-Hub/adapter-transformers.git to /tmp/pip-req-build-cajvfb7a
  Running command git clone -q https://github.com/Adapter-Hub/adapter-transformers.git /tmp/pip-req-build-cajvfb7a
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 895 kB 3.2 MB/s 
     |████████████████████████████████| 3.3 MB 38.0 MB/s 
     |████████████████████████████████| 43 kB 2.1 MB/s 
  Created wheel for adapter-transformers: filename=adapter_transformers-2.1.2-py3-none-any.whl size=2539220 sha256=2e6c41d3e2e85af0aff570cf4d9e5e7f2ea2a86042bdb33366d41ca59b15969c
  Stored in directory: /tmp/pip-ephem-wheel-cache-dgjozv8s/wheels/60/0e/d5/86e64fef3085f149b504f48af9ca6e8f6da427f42da9b4bbe5
Successfully built adapter-transformers
     |████████████████████████████████| 264 kB 5.2 MB/s 
     |████████████████████████████████| 243 kB 48.4 MB/s 
     |████████████████████████

In [ ]:
import torch
torch.cuda.is_available()

True

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive/", force_remount=True)

Mounted at /content/gdrive/


In [ ]:
import sys
sys.path.append('/content/gdrive/MyDrive/master_hpi/NLP_Project/code/')

In [ ]:
path = "/content/gdrive/MyDrive/master_hpi/NLP_Project/code/"

In [ ]:
path

'/content/gdrive/MyDrive/master_hpi/NLP_Project/code/'

In [ ]:
from datasets import load_dataset

dataset = load_dataset("sst")
dataset.num_rows


Downloading:   0%|          | 0.00/2.59k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.17k [00:00<?, ?B/s]

No config specified, defaulting to: sst/default


Downloading:   0%|          | 0.00/6.37M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/790k [00:00<?, ?B/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset sst downloaded and prepared to /root/.cache/huggingface/datasets/sst/default/1.0.0/b8a7889ef01c5d3ae8c379b84cc4080f8aad3ac2bc538701cbe0ac6416fb76ff. Subsequent calls will reuse this data.


{'test': 2210, 'train': 8544, 'validation': 1101}

In [ ]:
dataset['train'][0]





{'label': 0.6944400072097778,
 'sentence': "The Rock is destined to be the 21st Century 's new `` Conan '' and that he 's going to make a splash even greater than Arnold Schwarzenegger , Jean-Claud Van Damme or Steven Segal .",
 'tokens': "The|Rock|is|destined|to|be|the|21st|Century|'s|new|``|Conan|''|and|that|he|'s|going|to|make|a|splash|even|greater|than|Arnold|Schwarzenegger|,|Jean-Claud|Van|Damme|or|Steven|Segal|.",
 'tree': '70|70|68|67|63|62|61|60|58|58|57|56|56|64|65|55|54|53|52|51|49|47|47|46|46|45|40|40|41|39|38|38|43|37|37|69|44|39|42|41|42|43|44|45|50|48|48|49|50|51|52|53|54|55|66|57|59|59|60|61|62|63|64|65|66|67|68|69|71|71|0'}

In [ ]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

def encode_batch(batch):
  """Encodes a batch of input data using the model tokenizer."""
  return tokenizer(batch["sentence"], max_length=80, truncation=True, padding="max_length")


dataset = dataset.map(encode_batch, batched=True)

def label_mapping(batch):
  batch["labels"] = round(batch["label"])
  return batch
dataset = dataset.map(label_mapping)

dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

dataset["train"]["labels"]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

  0%|          | 0/9 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/3 [00:00<?, ?ba/s]

  0%|          | 0/8544 [00:00<?, ?ex/s]

  0%|          | 0/1101 [00:00<?, ?ex/s]

  0%|          | 0/2210 [00:00<?, ?ex/s]

tensor([1, 1, 1,  ..., 1, 0, 0])

In [ ]:
from transformers import AdapterConfig
from transformers import BertTokenizer, BertModelWithHeads

model = BertModelWithHeads.from_pretrained("bert-base-uncased", num_labels=2)

model.add_classification_head( 
    "sentiment/sst-2@ukp",
    num_labels=2,
    id2label={ 0: "👎", 1: "👍"}
     )


Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModelWithHeads: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModelWithHeads from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModelWithHeads from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
import numpy as np
from transformers import TrainingArguments, Trainer, EvalPrediction
path = "/content/gdrive/MyDrive/master_hpi/NLP_Project/code/"
import os

training_args = TrainingArguments(
    learning_rate=1e-4,
    num_train_epochs=6,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    logging_steps=200,
    remove_unused_columns=False,
    output_dir=path

)

def compute_accuracy(p: EvalPrediction):
  preds = np.argmax(p.predictions, axis=1)
  return {"acc": (preds == p.label_ids).mean()}


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["validation"],
    compute_metrics=compute_accuracy,
    do_save_full_model=True,
    
)
print(dataset["train"]["labels"])

tensor([1, 1, 1,  ..., 1, 0, 0])


In [ ]:
trainer.train()
print(dataset)

***** Running training *****
  Num examples = 8544
  Num Epochs = 6
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 1602


Step,Training Loss
200,0.454300
400,0.265500
600,0.176000
800,0.094400
1000,0.035200
1200,0.023500
1400,0.012900
1600,0.008000


Saving model checkpoint to /content/gdrive/MyDrive/master_hpi/NLP_Project/code/checkpoint-500
Configuration saved in /content/gdrive/MyDrive/master_hpi/NLP_Project/code/checkpoint-500/config.json
Model weights saved in /content/gdrive/MyDrive/master_hpi/NLP_Project/code/checkpoint-500/pytorch_model.bin
Saving model checkpoint to /content/gdrive/MyDrive/master_hpi/NLP_Project/code/checkpoint-1000
Configuration saved in /content/gdrive/MyDrive/master_hpi/NLP_Project/code/checkpoint-1000/config.json
Model weights saved in /content/gdrive/MyDrive/master_hpi/NLP_Project/code/checkpoint-1000/pytorch_model.bin
Saving model checkpoint to /content/gdrive/MyDrive/master_hpi/NLP_Project/code/checkpoint-1500
Configuration saved in /content/gdrive/MyDrive/master_hpi/NLP_Project/code/checkpoint-1500/config.json
Model weights saved in /content/gdrive/MyDrive/master_hpi/NLP_Project/code/checkpoint-1500/pytorch_model.bin


Training completed. Do not forget to share your model on huggingface.co/models =

DatasetDict({
    train: Dataset({
        features: ['attention_mask', 'input_ids', 'label', 'sentence', 'token_type_ids', 'tokens', 'tree', 'labels'],
        num_rows: 8544
    })
    validation: Dataset({
        features: ['attention_mask', 'input_ids', 'label', 'sentence', 'token_type_ids', 'tokens', 'tree', 'labels'],
        num_rows: 1101
    })
    test: Dataset({
        features: ['attention_mask', 'input_ids', 'label', 'sentence', 'token_type_ids', 'tokens', 'tree', 'labels'],
        num_rows: 2210
    })
})


In [ ]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 1101
  Batch size = 32


{'epoch': 6.0,
 'eval_acc': 0.8392370572207084,
 'eval_loss': 1.037583351135254,
 'eval_runtime': 4.0642,
 'eval_samples_per_second': 270.901,
 'eval_steps_per_second': 8.612}

In [ ]:
from transformers import TextClassificationPipeline

classifier = TextClassificationPipeline(model=model, tokenizer=tokenizer, device=0)

classifier("This is very great.")

[{'label': '👍', 'score': 0.9995078444480896}]

In [ ]:
print(model.active_adapters)
model.save_pretrained(path +"models/bert-base/", "bert-base-uncased")


Configuration saved in /content/gdrive/MyDrive/master_hpi/NLP_Project/code/models/bert-base/config.json


None


Model weights saved in /content/gdrive/MyDrive/master_hpi/NLP_Project/code/models/bert-base/pytorch_model.bin
